In [ ]:
import os
from openai import OpenAI
import openai
import json
from sqlalchemy.engine import create_engine
from sqlalchemy import text
import pandas as pd
import time
from pandas import DataFrame
import random

In [ ]:
"""A utility function to execute prompts against the LLM"""

def llm_model(prompt):
    client = OpenAI(api_key="")
    try:
        response =client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "system", "content": "You have an extensive knowledge on english and have an amazing skill to summarise documents without losing important information and delvireing the content in a very consize way"},
                      {"role": "user", "content": prompt}]
        )
        response_message = response.choices[0].message.content
        return response_message
    except openai.OpenAIError as e:
        print(f"OpenAI API error: {e}")
        return "Error with OpenAI API"
    except Exception as e:
        print(f"General error: {e}")
        return "General error in classification"

In [ ]:
"""A utility function to setup the connection to the realtime source"""

def create_kafka_engine(connection_params: str):
    
    connect_args = {
        "session_properties": {"streambased_connection": connection_params, "use_streambased": True},
        "http_scheme": "https",
        "schema": "streambased"
    }
    
    engine = create_engine("trino://streambased.cloud:8443/kafka", connect_args=connect_args)
    
    return engine

In [ ]:
"""Make a connection for the realtime source"""

connection_params = """{
  "bootstrap.servers": "pkc-l6wr6.europe-west2.gcp.confluent.cloud:9092",
  "security.protocol": "SASL_SSL",
  "sasl.jaas.config": "org.apache.kafka.common.security.plain.PlainLoginModule required username='4OOK2MI6TA37CC4K' password='0mzIsqc1Soz2thm9EutDuAkARY8CZqaNzJ5IF/KboX3l58XN/LZ+yQeuG3LmheIc';",
  "sasl.mechanism": "PLAIN",
  "schema.registry.url":"https://psrc-571d82.europe-west2.gcp.confluent.cloud",
  "basic.auth.credentials.source":"USER_INFO",
  "basic.auth.user.info":"FM36ASJ3BX2SWVXV:gKeS/p/CyfQYfd9/9diFwU6LsauvJ4mtmnPAv68ro7Bm81aZp29QOTXkV5yuv8af"
}"""

engine = create_kafka_engine(connection_params)
connection = engine.connect()

In [ ]:
"""Prepare 7 days worth of queries"""

current_timestamp = int(time.time()*1000)
day1_timestamp = current_timestamp - 86400000
day2_timestamp = day1_timestamp - 86400000
day3_timestamp = day2_timestamp - 86400000
day4_timestamp = day3_timestamp - 86400000
day5_timestamp = day4_timestamp - 86400000
day6_timestamp = day5_timestamp - 86400000
day7_timestamp = day6_timestamp - 86400000

"""We limit the rows returned for demo performance reasons"""

day1_query = f"SELECT * FROM yt_gaming_subtitles WHERE timestamp > {day1_timestamp} AND timestamp < {current_timestamp} LIMIT 5000" 
day2_query = f"SELECT * FROM yt_gaming_subtitles WHERE timestamp > {day1_timestamp} AND timestamp < {current_timestamp} LIMIT 5000"
day3_query = f"SELECT * FROM yt_gaming_subtitles WHERE timestamp > {day1_timestamp} AND timestamp < {current_timestamp} LIMIT 5000"
day4_query = f"SELECT * FROM yt_gaming_subtitles WHERE timestamp > {day1_timestamp} AND timestamp < {current_timestamp} LIMIT 5000"
day5_query = f"SELECT * FROM yt_gaming_subtitles WHERE timestamp > {day1_timestamp} AND timestamp < {current_timestamp} LIMIT 5000"
day6_query = f"SELECT * FROM yt_gaming_subtitles WHERE timestamp > {day1_timestamp} AND timestamp < {current_timestamp} LIMIT 5000"
day7_query = f"SELECT * FROM yt_gaming_subtitles WHERE timestamp > {day1_timestamp} AND timestamp < {current_timestamp} LIMIT 5000"


In [ ]:
"""Execute 7 days worth of queries - in production this would be executed asynchronously and continuously"""

connection = engine.connect()

rs = connection.execute(text(day1_query))
day1_l0_df = DataFrame(rs.fetchall())
day1_l0_df.columns = rs.keys()

In [ ]:
rs = connection.execute(text(day2_query))
day2_l0_df = DataFrame(rs.fetchall())
day2_l0_df.columns = rs.keys()

In [ ]:
rs = connection.execute(text(day3_query))
day3_l0_df = DataFrame(rs.fetchall())
day3_l0_df.columns = rs.keys()

In [ ]:
rs = connection.execute(text(day4_query))
day4_l0_df = DataFrame(rs.fetchall())
day4_l0_df.columns = rs.keys()

In [ ]:
rs = connection.execute(text(day5_query))
day5_l0_df = DataFrame(rs.fetchall())
day5_l0_df.columns = rs.keys()

In [ ]:
rs = connection.execute(text(day6_query))
day6_l0_df = DataFrame(rs.fetchall())
day6_l0_df.columns = rs.keys()

In [ ]:
rs = connection.execute(text(day7_query))
day7_l0_df = DataFrame(rs.fetchall())
day7_l0_df.columns = rs.keys()

In [ ]:
"""Level 1 summarisation: we take 1 day's worth of captions and summarise them to 1000 words"""

day2_l1_prompt = f"""I have provided the input of the youtube gaming subtitles for a single day please summmarise
it to 1000 words and make sure you don't leave important stuff out. The captions are: {' '.join(day2_l0_df["text"].dropna())}"""
day2_l1_summary = llm_model(day2_l1_prompt)

In [ ]:
day3_l1_prompt = f"""I have provided the input of the youtube gaming subtitles for a single day please summmarise
it to 1000 words and make sure you don't leave important stuff out. The captions are: {' '.join(day3_l0_df["text"].dropna())}"""
day3_l1_summary = llm_model(day3_l1_prompt)

In [ ]:
"""Level 2 summarisation: we take 1 day's worth of captions and extract 100 keywords"""

day4_l2_prompt = f"""I have provided the input of the youtube gaming subtitles for a single day please extract 100 keywords from it. The captions are: {' '.join(day4_l0_df["text"].dropna())}"""
day4_l2_summary = llm_model(day4_l2_prompt)

In [ ]:
day5_l2_prompt = f"""I have provided the input of the youtube gaming subtitles for a single day please extract 100 keywords from it. The captions are: {' '.join(day5_l0_df["text"].dropna())}"""
day5_l2_summary = llm_model(day5_l2_prompt)

In [ ]:
day6_l2_prompt = f"""I have provided the input of the youtube gaming subtitles for a single day please extract 100 keywords from it. The captions are: {' '.join(day6_l0_df["text"].dropna())}"""
day6_l2_summary = llm_model(day6_l2_prompt)

In [ ]:
day7_l2_prompt = f"""I have provided the input of the youtube gaming subtitles for a single day please extract 100 keywords from it. The captions are: {' '.join(day7_l0_df["text"].dropna())}"""
day7_l2_summary = llm_model(day7_l2_prompt)

In [ ]:
"""Now we can compose a prompt"""

prompt = f"""I have provided captions from Youtube gaming videos from the last day here: 

{' '.join(day1_l0_df["text"].dropna())}.

I have also provided a sumary of Youtube gaming videos from 24 to 72hrs ago here: 

{day2_l1_summary}
{day3_l1_summary}

I have also provided keywords extracted from Youtube gaming videos older than 72hrs here:

{day4_l2_summary}
{day5_l2_summary}
{day6_l2_summary}
{day7_l2_summary}

Use this information to answer the user's question. The user's question is what are the latest and enduring trends in the gaming field?"""

print(f"Composed a prompt with length: {len(prompt.split())}")


In [ ]:
"""Execute the prompt against the LLM"""

result = llm_model(prompt)
print(result)

In [ ]:
"""Let's compare with a similar prompt size with a lot less detail"""

top_k_selections = [day1_l0_df,day2_l0_df,day3_l0_df,day4_l0_df,day5_l0_df,day6_l0_df,day7_l0_df]
top_k_df_1 = random.choice(top_k_selections)
top_k_df_2 = random.choice(top_k_selections)

bad_prompt = f"""I have provided captions from Youtube gaming videos here: 

{' '.join(top_k_df_1["text"].dropna())}
{' '.join(top_k_df_2["text"].dropna())}

Use this information to answer the user's question. The user's question is what are the latest and enduring trends in the gaming field?"""

print(f"Composed a prompt with length: {len(bad_prompt.split())}")

In [ ]:
"""Execute the bad prompt against the LLM"""

bad_result = llm_model(bad_prompt)
print(bad_result)

In [ ]:
raw_prompt = "what are the latest and enduring trends in the gaming field?"
raw_result = llm_model(raw_prompt)

print(raw_result)